<a href="https://colab.research.google.com/github/ricardomsanti/douExplore/blob/master/DEX_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# PACKAGES

In [ ]:
!pip install html5lib
!pip install pyspark
!pip install PyPDF2
!pip install html5lib

# DRIVE MOUNT

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# IMPORTS

In [3]:
import os
from PyPDF2 import PdfReader
import pandas as pd
from wordcloud import WordCloud
import matplotlib.pyplot as plt


ModuleNotFoundError: No module named 'PyPDF2'

# SPARK


In [ ]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .master('local[*]') \
    .appName("my_app") \
    .getOrCreate()

import string
from string import punctuation
import pyspark.sql.functions as F

# DOU - UNIÃO

In [25]:
from references.paths import main_references as a
a.a

ModuleNotFoundError: No module named 'modules'

In [6]:
from REF import main_references as mr

FILES_PATH = mr.DRIVE_DOU_FED

FILES = [[x.name, \
          x.path]\
           for x in os.scandir(FILES_PATH) if ".pdf" in str(x)]

display(f"Total number of file: {len(FILES)}")

AttributeError: module 'REF.main_references' has no attribute 'DRIVE_DOU_FED'

In [ ]:
text_lines = []
#ARQUIVOS
for f in FILES:
  obj = PdfReader(open(f[1], 'rb'))
  ext_line = []
  #PAGINAS
  for page in range(0, len(obj.pages)):
    main_text = obj.pages[page].extract_text().split("\n")
    #LINHAS
    for line in range(0,len(main_text)):
      ext_line = []
      ext_line = [f[1], page, line, main_text[line]]
      print(f"Page >> {page}  \t" +
      f" Line >> {line}  \t" +
      f" Object Pages: >> {len(obj.pages)}")
      text_lines.append(ext_line)

In [ ]:

lines = pd.DataFrame(text_lines)
lines.columns = ["path", "pg_num","line", "string"]
data = spark.createDataFrame(lines)
data = data.withColumn("texto_regex", F.regexp_replace("string", "[\$#,\"!%&'()*+-./:;<=>?@^_`´{|}~\\\\]", ""))
data = data.withColumn("texto_trim", F.trim(data.texto_regex))
data = data.withColumn("trim_list", F.split("texto_trim", " "))
data.show(10, False)

In [ ]:
data2 = data.select("pg_num", "line", "string")
data2.show(100, False)

In [ ]:
WORDS = ["NACIONAL", "Ciência", "Abastecimento"]

data8 = data.select("pg_num", "trim_list")
for w in WORDS:  
  data8 = data8.withColumn(f"find_word >> {w}", F.array_contains(F.col("trim_list"), f"{w}"))
data8.show(10, False)


In [ ]:
data7 = data.withColumn("exp_line", F.explode("trim_list"))
data7= data7.withColumn("IDX", F.monotonically_increasing_id())
data7 = data7.select("exp_line", "IDX").drop_duplicates()
 
data7.show(10)

In [ ]:
data3 = data2.select("exp_line")
data4 = data3.drop_duplicates()
data4 = data4.withColumn("IDX", F.monotonically_increasing_id())